<a href="https://colab.research.google.com/github/antfolk/BMEN35_2023/blob/main/Session6/BMEN35_ensamble_cinc_assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6
## Fill in your name below
Albert Ahnlide

## Your mission is now the following:

You will use data from the Computing in Cardiology challenge 2022 (as was explained in the lectures) (https://moody-challenge.physionet.org/2022/). The training set contains data from 942 patients.

We have done some preprocessing of the data so you have features and labels for both **murmur** and **outcome**. The features include age, sex, weight, heigth, pregnancy status and mean, variance and skewness for the phonocardiogram at five different locations.

Evaluate different ensamble methods (at least 3) from sklearn (https://scikit-learn.org/stable/modules/ensemble.html and https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) and see how they perform on the CinC2022 challenge data. Take some time to read the documentation and see what options are available.

As you may remember from the lecture there is quite an eloborate scoring scheme, however this is handled by the file cinc2022metric.py and the methods contained therein.

**Also remeber you have two sets of labels!!! One set of labels for murmurs (Present, Unknown, Absent) and one for outcomes (Abnormal, Normal).**

**Another thing you need to take into account is that the scoring functions need label probabilities of the predicted classes.**

You will also need one-hot encoding of "hard" values for the training labels and for the test labels.

We will start by uploading some files. You need to upload the files cinc2022metrics.py, feats.csv, murmur_labels.csv and outcome_labels.csv

In [5]:
#from google.colab import files
#_ = files.upload() # Upload the other files available in github under Session 6

Next we will import some of the libraries/modules needed. (You will need to import others later on).

In [6]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import cinc2022metrics as cm # Our own little metrics file
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

/Users/albertahnlide/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Next we will import data and the two different sets of labels and switch to numpy arrays.



In [7]:
feats = pd.read_csv('feats.csv', header=None)
murmur_labels = pd.read_csv('murmur_labels.csv', header=None)
outcome_labels = pd.read_csv('outcome_labels.csv', header=None)

feats = feats.to_numpy()
murmur_labels = murmur_labels.to_numpy()
outcome_labels = outcome_labels.to_numpy()

Here we will split the data and also define what the different classes for murmur and outcome are.

In [8]:
X_train, X_test, y_train_murmur, y_test_murmur, y_train_outcome, y_test_outcome = train_test_split(feats, murmur_labels, outcome_labels, test_size=0.2, random_state=0)
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

Next you can try out some of the available ensemble methods. Remember you need to predict probabilities for both classifiers.

In [14]:
# Do some imports (you now how to do this by now)
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier



rfMur = RandomForestClassifier(n_estimators=100)
rfOut = RandomForestClassifier(n_estimators=100)
rfMur.fit(X_train, y_train_murmur)
rfOut.fit(X_train, y_train_outcome)
rfMurProb = rfMur.predict_proba(X_test)
rfOutProb = rfOut.predict_proba(X_test)

gbMur = GradientBoostingClassifier(n_estimators=100)
gbOut = GradientBoostingClassifier(n_estimators=100)
gbMur.fit(X_train, y_train_murmur)
gbOut.fit(X_train, y_train_outcome)
gbMurProb = gbMur.predict_proba(X_test)
gbOutProb = gbOut.predict_proba(X_test)

adaMur = AdaBoostClassifier(n_estimators=100)
adaOut = AdaBoostClassifier(n_estimators=100)
adaMur.fit(X_train, y_train_murmur)
adaOut.fit(X_train, y_train_outcome)
adaMurProb = adaMur.predict_proba(X_test)
adaOutProb = adaOut.predict_proba(X_test)

bagMur = BaggingClassifier(n_estimators=100)
bagOut = BaggingClassifier(n_estimators=100)
bagMur.fit(X_train, y_train_murmur)
bagOut.fit(X_train, y_train_outcome)
bagMurProb = bagMur.predict_proba(X_test)
bagOutProb = bagOut.predict_proba(X_test)



score_rfMur = rfMur.score(X_test,y_test_murmur)
score_rfOut = rfOut.score(X_test, y_test_outcome)
print("rf murmur: " + str(score_rfMur))
print("rf outcome: " + str(score_rfOut))

score_gbMur = gbMur.score(X_test, y_test_murmur)
score_gbOut = gbOut.score(X_test,y_test_outcome)
print("gb murmur: " + str(score_gbMur))
print("gb outcome: " + str(score_gbOut))

score_adaMur = adaMur.score(X_test, y_test_murmur)
score_adaOut = adaOut.score(X_test,y_test_outcome)
print("adaboost murmur: " + str(score_adaMur))
print("adaboost outcome: " + str(score_adaOut))

score_bagMur = bagMur.score(X_test, y_test_murmur)
score_bagOut = bagOut.score(X_test,y_test_outcome)
print("bagging murmur: " + str(score_bagMur))
print("bagging outcome: " + str(score_bagOut))




/Users/albertahnlide/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/albertahnlide/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/albertahnlide/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/albertahnlide/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y wa

rf murmur: 0.7037037037037037
rf outcome: 0.6031746031746031
gb murmur: 0.7037037037037037
gb outcome: 0.6296296296296297
adaboost murmur: 0.6455026455026455
adaboost outcome: 0.6031746031746031
bagging murmur: 0.7037037037037037
bagging outcome: 0.6031746031746031


In [25]:
y_pred_murmur_prob = gbMurProb
y_pred_outcome_prob = gbOutProb


ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_test_murmur_bin = ohe.fit_transform(y_test_murmur) 
y_test_outcome_bin = ohe.fit_transform(y_test_outcome)

/Users/albertahnlide/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/albertahnlide/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Now you should also One-hot encode the **test** (not the predicted ones) labels for both murmur and outcome.

In [26]:
murmur_scores, outcome_scores = cm.compute_scores(y_test_murmur_bin,   # One-hot encoded test labels for murmur eg. [1 0 0]
                                                  y_pred_murmur_prob,  # One-hot encoded predicted probabilities for murmur eg. [0.1 0.7 0.2]
                                                  murmur_classes,      # As defined before
                                                  y_test_outcome_bin,  # One-hot encoded test labels for outcome eg. [1 0]
                                                  y_pred_outcome_prob, # One-hot encoded predicted probabilities for outcome eg. [0.1 0.9]
                                                  outcome_classes)     # As defined before



Now we have calculated a whole bunch of scores for both murmur and outcome. We can print them using the following.

In [27]:
cm.print_scores(murmur_scores, outcome_scores)

#Murmur scores
AUROC,AUPRC,F-meas,Accuracy,Weighted Accuracy,Cost
0.543,0.364,0.305,0.704,0.649,13907.760

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.650,0.638,0.628,0.630,0.629,11778.128

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.538,0.551,0.540
AUPRC,0.269,0.741,0.082
F-measure,0.089,0.826,0.000
Accuracy,0.050,0.963,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.650,0.650
AUPRC,0.616,0.660
F-measure,0.607,0.650
Accuracy,0.628,0.631



Did you manage to comparable scores to those in the leaderboard as shown in the last slide of the lecture? (It should be noted that we don't have the validation data so we are perhaps comparing apples and oranges). Which set of ensemble classifiers worked the best?

    I did unfortunatelly not get as good results as the ones that led to the scores on the leaderboard. The Random Forest got the best score on the murmur while the Gradient Boosting got the best on outcome and it also had the minimum cost. 

## The End